Colab을 활용한 deep learning model inference

*   transformer model
*   BERT, GPT, 이미지 분류 및 인식 모델 등의 기반


- https://colab.research.google.com/github/huggingface/notebooks/blob/main/transformers_doc/en/pytorch/translation.ipynb

In [ ]:
# Transformers 설치 방법
! pip install transformers datasets evaluate sacrebleu

# 번역[[translation]]

번역은 한 언어로 된 시퀀스를 다른 언어로 변환합니다. 번역이나 요약은 입력을 받아 일련의 출력을 반환하는 강력한 프레임워크인 시퀀스-투-시퀀스 문제로 구성할 수 있는 대표적인 태스크입니다. 번역 시스템은 일반적으로 다른 언어로 된 텍스트 간의 번역에 사용되지만, 음성 간의 통역이나 텍스트-음성 또는 음성-텍스트와 같은 조합에도 사용될 수 있습니다.

학습할 내용은:

1. 영어 텍스트를 프랑스어로 번역하기 위해 [T5](https://huggingface.co/t5-small) 모델을 OPUS Books 데이터세트의 영어-프랑스어 하위 집합으로 파인튜닝하는 방법과
2. 파인튜닝된 모델을 추론에 사용하는 방법입니다.

<Tip>
이 태스크 가이드는 아래 모델 아키텍처에도 응용할 수 있습니다.

<!--This tip is automatically generated by `make fix-copies`, do not fill manually!-->

[BART](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/bart), [BigBird-Pegasus](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/bigbird_pegasus), [Blenderbot](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/blenderbot), [BlenderbotSmall](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/blenderbot-small), [Encoder decoder](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/encoder-decoder), [FairSeq Machine-Translation](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/fsmt), [GPTSAN-japanese](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/gptsan-japanese), [LED](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/led), [LongT5](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/longt5), [M2M100](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/m2m_100), [Marian](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/marian), [mBART](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/mbart), [MT5](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/mt5), [MVP](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/mvp), [NLLB](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/nllb), [NLLB-MOE](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/nllb-moe), [Pegasus](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/pegasus), [PEGASUS-X](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/pegasus_x), [PLBart](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/plbart), [ProphetNet](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/prophetnet), [SwitchTransformers](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/switch_transformers), [T5](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/t5), [XLM-ProphetNet](https://huggingface.co/docs/transformers/main/ko/tasks/../model_doc/xlm-prophetnet)

<!--End of the generated tip-->


## OPUS Books 데이터세트 가져오기[[load-opus-books-dataset]]

먼저 🤗 Datasets 라이브러리에서 [OPUS Books](https://huggingface.co/datasets/opus_books) 데이터세트의 영어-프랑스어 하위 집합을 가져오세요.

In [ ]:
from datasets import load_dataset

books = load_dataset("opus_books", "en-fr")

데이터세트를 `train_test_split` 메서드를 사용하여 훈련 및 테스트 데이터로 분할하세요.

In [ ]:
books = books["train"].train_test_split(test_size=0.001) # 0.2 -> 0.001

훈련 데이터에서 예시를 살펴볼까요?

In [ ]:
books["train"][123]

반환된 딕셔너리의 `translation` 키가 텍스트의 영어, 프랑스어 버전을 포함하고 있는 것을 볼 수 있습니다.

## 전처리[[preprocess]]

다음 단계로 영어-프랑스어 쌍을 처리하기 위해 T5 토크나이저를 가져오세요.

In [ ]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


만들 전처리 함수는 아래 요구사항을 충족해야 합니다:

1. T5가 번역 태스크임을 인지할 수 있도록 입력 앞에 프롬프트를 추가하세요. 여러 NLP 태스크를 할 수 있는 모델 중 일부는 이렇게 태스크 프롬프트를 미리 줘야합니다.
2. ~원어(영어)과 번역어(프랑스어)를 별도로 토큰화하세요. 영어 어휘로 사전 학습된 토크나이저로 프랑스어 텍스트를 토큰화할 수는 없기 때문입니다.~
3. `max_length` 매개변수로 설정한 최대 길이보다 길지 않도록 시퀀스를 truncate하세요.

In [ ]:
source_lang = "en"
target_lang = "es"
prefix = "translate English to French: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

truncation : max_length 128이상은 버림

전체 데이터세트에 전처리 함수를 적용하려면 🤗 Datasets의 `map` 메서드를 사용하세요. `map` 함수의 속도를 높이려면 `batched=True`를 설정하여 데이터세트의 여러 요소를 한 번에 처리하는 방법이 있습니다.

In [ ]:
tokenized_books = books.map(preprocess_function, batched=True)

이제 `DataCollatorForSeq2Seq`를 사용하여 학습용 배치를 생성합니다. 데이터세트의 최대 길이로 전부를 padding하는 대신, 데이터 정렬 중 각 배치의 최대 길이로 문장을 *동적으로 padding*하는 것이 더 효율적입니다.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## 평가[[evalulate]]

훈련 중에 메트릭을 포함하면 모델의 성능을 평가하는 데 도움이 됩니다. 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) 라이브러리로 평가 방법(evaluation method)을 빠르게 가져올 수 있습니다. 현재 태스크에 적합한 SacreBLEU 메트릭을 가져오세요. (메트릭을 가져오고 계산하는 방법에 대해 자세히 알아보려면 🤗 Evaluate [둘러보기](https://huggingface.co/docs/evaluate/a_quick_tour)를 참조하세요):

In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

SacreBLEU 점수를 계산하는 함수를 생성하세요.  

In [ ]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

이제 `compute_metrics` 함수는 준비되었고, 훈련 과정을 설정할 때 다시 살펴볼 예정입니다.

## 훈련[[train]]

<Tip>

`Trainer`로 모델을 파인튜닝하는 방법에 익숙하지 않다면 [여기](https://huggingface.co/docs/transformers/main/ko/tasks/../training#train-with-pytorch-trainer)에서 기본 튜토리얼을 살펴보시기 바랍니다!

</Tip>

모델을 훈련시킬 준비가 되었군요! `AutoModelForSeq2SeqLM`으로 T5를 로드하세요:

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

이제 세 단계만 거치면 끝입니다:

1. `Seq2SeqTrainingArguments`에서 훈련 하이퍼파라미터를 정의하세요. 유일한 필수 매개변수는 모델을 저장할 위치인 `output_dir`입니다. 모델을 Hub에 푸시하기 위해 `push_to_hub=True`로 설정하세요. (모델을 업로드하려면 Hugging Face에 로그인해야 합니다.) `Trainer`는 에폭이 끝날때마다 SacreBLEU 메트릭을 평가하고 훈련 체크포인트를 저장합니다.
2. `Seq2SeqTrainer`에 훈련 인수를 전달하세요. 모델, 데이터 세트, 토크나이저, data collator 및 `compute_metrics` 함수도 덩달아 전달해야 합니다.
3. `train()`을 호출하여 모델을 파인튜닝하세요.

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_opus_books_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    max_steps=10,                       # gpu 계산량을 줄이기 위해서 추가
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


## 추론[[inference]]

이제 모델을 파인튜닝했으니 추론에 사용할 수 있습니다!

다른 언어로 번역하고 싶은 텍스트를 써보세요. T5의 경우 원하는 태스크를 입력의 접두사로 추가해야 합니다. 예를 들어 영어에서 프랑스어로 번역하는 경우, 아래와 같은 접두사가 추가됩니다:

In [ ]:
# text = "translate English to French: Legumes share resources with nitrogen-fixing bacteria."
text = "translate English to French: Would you like a cup of coffee?"

텍스트를 토큰화하고 `input_ids`를 PyTorch 텐서로 반환하세요:  

In [ ]:
# inputs = tokenizer(text, return_tensors="pt").input_ids
inputs = tokenizer(text, return_tensors="pt").input_ids.to("cuda:0")

In [ ]:
print(inputs)

tensor : 자료형 중 하나 (list, set, tuple, dic... str, num)  
행렬(matrix)을 n 차원 한거

`generate()` 메서드로 번역을 생성하세요. 다양한 텍스트 생성 전략 및 생성을 제어하기 위한 매개변수에 대한 자세한 내용은 [Text Generation](https://huggingface.co/docs/transformers/main/ko/tasks/../main_classes/text_generation) API를 살펴보시기 바랍니다.

In [ ]:
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)

In [ ]:
print(outputs)

생성된 토큰 ID들을 다시 텍스트로 디코딩하세요:

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)